# Check colab

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2ef64152-6479-5726-38d4-03f1c14b10fe)


In [4]:
!nvidia-smi

Sat Feb  6 06:02:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil -q
!pip install psutil -q
!pip install humanize -q
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.8 GB  | Proc size: 112.1 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [6]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import time, psutil
start = time.time()- psutil.boot_time()
left= 12*3600 - start
print('Time remaining for this session is: ', left/3600)

Found GPU at: /device:GPU:0
Time remaining for this session is:  11.96003582669629


# Prepare dataset and repo

In [7]:
# Clone repo
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2399, done.
remote: Counting objects: 100% (2399/2399), done.
remote: Compressing objects: 100% (1997/1997), done.
remote: Total 2399 (delta 573), reused 1395 (delta 375), pack-reused 0
Receiving objects: 100% (2399/2399), 30.77 MiB | 37.83 MiB/s, done.
Resolving deltas: 100% (573/573), done.


In [8]:
# Copy dataset
!cp -r /content/drive/MyDrive/DATASET/mini-furniture-tfrecords/. /content/dataset/

In [9]:
test_record_fname = '/content/dataset/validation.record'
train_record_fname = '/content/dataset/train.record'
label_map_pbtxt_fname = '/content/dataset/label_map.pbtxt'

# Install TF OD

In [10]:
import os
import tensorflow as tf
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
tf.get_logger().setLevel('ERROR')          

In [11]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install . -q

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.27.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


In [12]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [13]:
# TEST
%%bash
cd /content/models/research
python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 3.6s
I0206 06:05:36.299129 139892111091584 test_util.py:2076] time(__main__.ModelBuilderTF2Test.test_create_center_net_model): 3.6s
[       OK ] ModelBuilderTF2Test.test_create_center_net_model
[ RUN      ] ModelBuilderTF2Test.test_create_experimental_model
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
I0206 06:05:36.300222 139892111091584 test_util.py:2076] time(__main__.ModelBuilderTF2Test.test_create_experimental_model): 0.0s
[       OK ] ModelBuilderTF2Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF2Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)): 0.04s
I0206 06:05:36.343162 139892111091584 test_util.py:2

# Prepare Training

In [14]:
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 4
    }
}


#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'efficientdet-d0'

num_steps = 100000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [15]:
#download pretrained weights
%mkdir /content/models/research/deploy/
%cd /content/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/content/models/research/deploy
--2021-02-06 06:06:00--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.8.16, 2607:f8b0:4004:803::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.8.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M  --.-KB/s    in 0.1s    

2021-02-06 06:06:00 (289 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



In [16]:
#download base training configuration file
%cd /content/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/models/research/deploy
--2021-02-06 06:06:01--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config’

ssd_efficientdet_d0 100%[===================>]   4.52K  --.-KB/s    in 0s      

2021-02-06 06:06:01 (68.8 MB/s) - ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config’ saved [4630/4630]



In [17]:
# prepare
pipeline_fname = '/content/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'
# fine_tune_checkpoint = "/content/drive/MyDrive/COLAB PROJECTS/learn-tf-object-detection/training-d0/ckpt-8"

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=10, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [18]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    

    s = re.sub('learning_rate_base: [0-9]+e-2',
               'learning_rate_base: {}'.format(0.000008), s)
    
    s = re.sub('warmup_learning_rate: .[0-9]+',
               'warmup_learning_rate: {}'.format(.0000001), s)

    f.write(s)

/content/models/research/deploy
writing custom configuration file


In [19]:
!cat /content/models/research/deploy/pipeline_file.config

 # SSD with EfficientNet-b0 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d0).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b0 checkpoint.
#
# Train on TPU-8

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 10
    add_background_class: false
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      

# Training

In [ ]:
# !mkdir /content/drive/MyDrive/COLAB\ PROJECTS/furniture-detection/effdet-d0/

In [20]:
# directory to save model
model_dir = "/content/drive/MyDrive/COLAB\ PROJECTS/furniture-detection/effdet-d0/"
pipeline_file = '/content/models/research/deploy/pipeline_file.config'

In [21]:
model_dir

'/content/drive/MyDrive/COLAB\\ PROJECTS/furniture-detection/effdet-d0/'

## History

In [ ]:
# learning_rate_base 0.08
# warmup_learning_rate .0001
# loss 1.048
# step 2500
# mAP 0.037889
# ckpt-6
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=2500 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 10:10:00.071979 140548343969664 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2500
I0203 10:10:00.076714 140548343969664 config_util.py:552] Maybe overwriting train_steps: 2500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 10:10:00.076886 140548343969664 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 10:10:00.096030 140548343969664 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 10:10:00.096145 140548343969664 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 10:10:00.096216 140548343969664 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 10:10:00.104894 140548343969664 efficientnet_model.py

In [ ]:
# learning_rate_base 0.008
# warmup_learning_rate .00001
# loss 1.008
# step 5000
# mAP 0.127852
# total_loss: 1.019979
# ckpt-11
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=5000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 10:51:42.763015 139629557430144 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0203 10:51:42.768066 139629557430144 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 10:51:42.768239 139629557430144 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 10:51:42.787176 139629557430144 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 10:51:42.787293 139629557430144 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 10:51:42.787371 139629557430144 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 10:51:42.796274 139629557430144 efficientnet_model.py

In [ ]:
# learning_rate_base 0.008
# warmup_learning_rate .00001
# loss 1.031
# step 7500
# mAP 0.136
# total_loss: 1.013699
# ckpt-16
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=7500 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 11:34:18.775930 140008747947904 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 7500
I0203 11:34:18.780910 140008747947904 config_util.py:552] Maybe overwriting train_steps: 7500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 11:34:18.781081 140008747947904 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 11:34:18.800170 140008747947904 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 11:34:18.800290 140008747947904 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 11:34:18.800364 140008747947904 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 11:34:18.809194 140008747947904 efficientnet_model.py

In [ ]:
# learning_rate_base 0.008
# warmup_learning_rate .00001
# loss 0.724
# step 10000
# mAP 0.133142
# total_loss: 1.004282
# ckpt-21
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 12:26:59.102168 140563513374592 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 10000
I0203 12:26:59.106977 140563513374592 config_util.py:552] Maybe overwriting train_steps: 10000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 12:26:59.107142 140563513374592 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 12:26:59.125818 140563513374592 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 12:26:59.125940 140563513374592 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 12:26:59.126011 140563513374592 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 12:26:59.134657 140563513374592 efficientnet_model.

In [ ]:
# learning_rate_base 0.008
# warmup_learning_rate .00001
# loss 0.838
# step 12500
# mAP 0.136514
# total_loss: 1.070963
# ckpt-26
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=12500 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 13:22:20.596782 140468223137664 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 12500
I0203 13:22:20.601789 140468223137664 config_util.py:552] Maybe overwriting train_steps: 12500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 13:22:20.601971 140468223137664 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 13:22:20.621268 140468223137664 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 13:22:20.621386 140468223137664 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 13:22:20.621462 140468223137664 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 13:22:20.630457 140468223137664 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.791
# step 15000
# mAP 0.149486
# total_loss: 1.022574
# ckpt-31
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=15000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 14:14:55.089340 139633077249920 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 15000
I0203 14:14:55.094162 139633077249920 config_util.py:552] Maybe overwriting train_steps: 15000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 14:14:55.094324 139633077249920 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 14:14:55.113612 139633077249920 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 14:14:55.113725 139633077249920 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 14:14:55.113798 139633077249920 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 14:14:55.122615 139633077249920 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.857
# step 17500
# mAP 0.151670
# total_loss: 1.062866
# ckpt-36
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=17500 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 15:06:46.782424 140358886299520 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 17500
I0203 15:06:46.787490 140358886299520 config_util.py:552] Maybe overwriting train_steps: 17500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 15:06:46.787678 140358886299520 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 15:06:46.806952 140358886299520 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 15:06:46.807071 140358886299520 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 15:06:46.807147 140358886299520 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 15:06:46.815930 140358886299520 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.822
# step 20000
# mAP 0.154371
# total_loss: 1.062066
# ckpt-41
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=20000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0203 15:49:26.010005 139823788349312 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 20000
I0203 15:49:26.014639 139823788349312 config_util.py:552] Maybe overwriting train_steps: 20000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0203 15:49:26.014788 139823788349312 config_util.py:552] Maybe overwriting use_bfloat16: False
I0203 15:49:26.033030 139823788349312 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0203 15:49:26.033154 139823788349312 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0203 15:49:26.033225 139823788349312 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0203 15:49:26.041844 139823788349312 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.762
# step 25000
# mAP 0.161853
# total_loss: 1.064271
# ckpt-51
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=25000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 08:18:27.230503 140428971984768 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 25000
I0204 08:18:27.235509 140428971984768 config_util.py:552] Maybe overwriting train_steps: 25000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 08:18:27.235684 140428971984768 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 08:18:27.254809 140428971984768 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 08:18:27.254949 140428971984768 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 08:18:27.255025 140428971984768 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 08:18:27.268243 140428971984768 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.712
# step 30000
# mAP 0.158
# total_loss: 1.087535
# ckpt-61
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=30000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 09:32:02.616400 140047158912896 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 30000
I0204 09:32:02.621016 140047158912896 config_util.py:552] Maybe overwriting train_steps: 30000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 09:32:02.621169 140047158912896 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 09:32:02.640062 140047158912896 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 09:32:02.640183 140047158912896 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 09:32:02.640263 140047158912896 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 09:32:02.649344 140047158912896 efficientnet_model.

In [ ]:
# learning_rate_base 0.0008
# warmup_learning_rate .000001
# loss 0.759
# step 35000
# mAP 0.163
# total_loss: 1.107908
# ckpt-71
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=35000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 11:05:06.328940 139723028653952 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 35000
I0204 11:05:06.333631 139723028653952 config_util.py:552] Maybe overwriting train_steps: 35000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 11:05:06.333801 139723028653952 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 11:05:06.352441 139723028653952 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 11:05:06.352556 139723028653952 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 11:05:06.352629 139723028653952 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 11:05:06.361442 139723028653952 efficientnet_model.

In [ ]:
# learning_rate_base 0.0004
# warmup_learning_rate .000001
# loss 0.723
# step 40000
# mAP 0.169
# total_loss: 1.108755
# ckpt-81
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=40000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 12:20:03.313590 139903627544448 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 40000
I0204 12:20:03.318569 139903627544448 config_util.py:552] Maybe overwriting train_steps: 40000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 12:20:03.318747 139903627544448 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 12:20:03.338046 139903627544448 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 12:20:03.338167 139903627544448 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 12:20:03.338251 139903627544448 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 12:20:03.347436 139903627544448 efficientnet_model.

In [ ]:
# learning_rate_base 0.00008
# warmup_learning_rate .0000001
# loss 0.719
# step 45000
# mAP 0.166
# total_loss: 1.132968
# ckpt-90
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=45000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 13:53:00.944190 139706221045632 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 45000
I0204 13:53:00.948929 139706221045632 config_util.py:552] Maybe overwriting train_steps: 45000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 13:53:00.949103 139706221045632 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 13:53:00.968253 139706221045632 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 13:53:00.968376 139706221045632 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 13:53:00.968449 139706221045632 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 13:53:00.977293 139706221045632 efficientnet_model.

In [ ]:
# learning_rate_base 0.00008
# warmup_learning_rate .0000001
# loss 0.931
# step 50000
# mAP 0.163768
# total_loss: 1.131024
# ckpt-101
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=50000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0204 15:04:28.131883 140718279014272 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 50000
I0204 15:04:28.136864 140718279014272 config_util.py:552] Maybe overwriting train_steps: 50000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0204 15:04:28.137039 140718279014272 config_util.py:552] Maybe overwriting use_bfloat16: False
I0204 15:04:28.161435 140718279014272 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0204 15:04:28.161558 140718279014272 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0204 15:04:28.161625 140718279014272 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0204 15:04:28.170824 140718279014272 efficientnet_model.

In [ ]:
# learning_rate_base 0.00008
# warmup_learning_rate .0000001
# loss 0.513
# step 55000
# mAP 0.168743
# total_loss: 1.134390
# ckpt-111
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=55000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 02:59:33.664420 140006097209216 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 55000
I0205 02:59:33.669456 140006097209216 config_util.py:552] Maybe overwriting train_steps: 55000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 02:59:33.669615 140006097209216 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 02:59:33.688588 140006097209216 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 02:59:33.688716 140006097209216 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 02:59:33.688789 140006097209216 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 02:59:33.697468 140006097209216 efficientnet_model.

In [ ]:
# learning_rate_base 0.00008
# warmup_learning_rate .0000001
# loss 0.880
# step 65000
# mAP 0.167872
# total_loss: 1.135594
# ckpt-131
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=65000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 07:20:21.968583 139685730051968 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 65000
I0205 07:20:21.973672 139685730051968 config_util.py:552] Maybe overwriting train_steps: 65000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 07:20:21.973844 139685730051968 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 07:20:21.993100 139685730051968 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 07:20:21.993223 139685730051968 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 07:20:21.993299 139685730051968 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 07:20:22.002217 139685730051968 efficientnet_model.

In [ ]:
# learning_rate_base 0.00008
# warmup_learning_rate .0000001
# loss 0.630
# step 70000
# mAP 0.166565
# total_loss: 1.142865
# ckpt-138
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=70000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 09:13:53.452776 140037606623104 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 70000
I0205 09:13:53.457507 140037606623104 config_util.py:552] Maybe overwriting train_steps: 70000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 09:13:53.457669 140037606623104 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 09:13:53.478416 140037606623104 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 09:13:53.478547 140037606623104 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 09:13:53.478613 140037606623104 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 09:13:53.487690 140037606623104 efficientnet_model.

In [ ]:
# learning_rate_base 0.00004
# warmup_learning_rate .0000001
# loss 0.754
# step 75000
# mAP 0.163307
# total_loss: 1.144048
# ckpt-151
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=75000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 09:59:07.171824 140001441867648 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 75000
I0205 09:59:07.176342 140001441867648 config_util.py:552] Maybe overwriting train_steps: 75000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 09:59:07.176494 140001441867648 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 09:59:07.194818 140001441867648 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 09:59:07.194932 140001441867648 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 09:59:07.195003 140001441867648 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 09:59:07.203949 140001441867648 efficientnet_model.

In [ ]:
# learning_rate_base 0.00002
# warmup_learning_rate .0000001
# loss 0.979
# step 80000
# mAP 0.171488
# total_loss: 1.141819
# ckpt-161
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=80000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 11:10:34.413178 140582993913728 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 80000
I0205 11:10:34.418038 140582993913728 config_util.py:552] Maybe overwriting train_steps: 80000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 11:10:34.418216 140582993913728 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 11:10:34.437222 140582993913728 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 11:10:34.437336 140582993913728 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 11:10:34.437407 140582993913728 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 11:10:34.446232 140582993913728 efficientnet_model.

In [ ]:
# learning_rate_base 0.00002
# warmup_learning_rate .0000001
# loss 0.649
# step 85000
# mAP 0.171648
# total_loss: 1.145144
# ckpt-171
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=85000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 12:59:20.904843 139753553041280 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 85000
I0205 12:59:20.909373 139753553041280 config_util.py:552] Maybe overwriting train_steps: 85000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 12:59:20.909539 139753553041280 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 12:59:20.927987 139753553041280 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 12:59:20.928105 139753553041280 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 12:59:20.928176 139753553041280 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 12:59:20.937016 139753553041280 efficientnet_model.

In [ ]:
# learning_rate_base 0.000008
# warmup_learning_rate .0000001
# loss 0.782
# step 90000
# mAP 0.171369
# total_loss: 1.138356
# ckpt-181
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=90000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0205 14:11:17.350030 139644280031104 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 90000
I0205 14:11:17.354903 139644280031104 config_util.py:552] Maybe overwriting train_steps: 90000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0205 14:11:17.355067 139644280031104 config_util.py:552] Maybe overwriting use_bfloat16: False
I0205 14:11:17.373228 139644280031104 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0205 14:11:17.373359 139644280031104 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0205 14:11:17.373426 139644280031104 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0205 14:11:17.382391 139644280031104 efficientnet_model.

In [24]:
# learning_rate_base 0.000008
# warmup_learning_rate .0000001
# loss 0.726
# step 100000
# mAP 0.172042
# total_loss: 1.142126
# ckpt-201
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=100000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0206 07:15:42.573434 139662294366080 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 100000
I0206 07:15:42.578225 139662294366080 config_util.py:552] Maybe overwriting train_steps: 100000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0206 07:15:42.578432 139662294366080 config_util.py:552] Maybe overwriting use_bfloat16: False
I0206 07:15:42.597573 139662294366080 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0206 07:15:42.597718 139662294366080 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0206 07:15:42.597799 139662294366080 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0206 07:15:42.606657 139662294366080 efficientnet_mode

In [38]:
# learning_rate_base 0.000008
# warmup_learning_rate .0000001
# loss 0.689
# step 110000
# mAP 0.171326
# total_loss: 1.145587
# ckpt-221
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=110000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0206 08:38:04.639199 139658280060800 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 110000
I0206 08:38:04.643890 139658280060800 config_util.py:552] Maybe overwriting train_steps: 110000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0206 08:38:04.644072 139658280060800 config_util.py:552] Maybe overwriting use_bfloat16: False
I0206 08:38:04.662972 139658280060800 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0206 08:38:04.663106 139658280060800 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0206 08:38:04.663199 139658280060800 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0206 08:38:04.672062 139658280060800 efficientnet_mode

## Present

In [40]:
# learning_rate_base 0.000008
# warmup_learning_rate .0000001
# loss 0.689
# step 110000
# mAP 0.171326
# total_loss: 1.145587
# ckpt-221
!python /content/models/research/object_detection/model_main_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --model_dir={model_dir} \
  --alsologtostderr \
  --num_train_steps=115000 \
  --sample_1_of_n_eval_examples=1 \
  --checkpoint_every_n=500\
  --num_eval_steps=500

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0206 11:04:57.504786 140516805912448 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 115000
I0206 11:04:57.509840 140516805912448 config_util.py:552] Maybe overwriting train_steps: 115000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0206 11:04:57.510008 140516805912448 config_util.py:552] Maybe overwriting use_bfloat16: False
I0206 11:04:57.529552 140516805912448 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0206 11:04:57.529664 140516805912448 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0206 11:04:57.529752 140516805912448 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0206 11:04:57.538638 140516805912448 efficientnet_mode

# Evaluation

In [41]:
# I0202 01:06:58.832048 140411743831936 model_lib_v2.py:651] Step 6000 per-step time 1.314s loss=0.768
!python /content/models/research/object_detection/model_main_tf2.py --model_dir={model_dir} \
     --pipeline_config_path={pipeline_file} \
     --checkpoint_dir={model_dir}

W0206 12:09:51.803311 140528935495552 model_lib_v2.py:1034] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0206 12:09:51.803523 140528935495552 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0206 12:09:51.803622 140528935495552 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0206 12:09:51.803712 140528935495552 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0206 12:09:51.803844 140528935495552 model_lib_v2.py:1049] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
I0206 12:09:56.188574 140528935495552 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0206 12:09:56.188751 140528935495552 ssd_efficientnet_b

# Save model

In [26]:
# model_dir = "/content/drive/MyDrive/COLAB\ PROJECTS/learn-tf-object-detection/training-d0/"

In [27]:
#see where our model saved weights
%ls {model_dir}

checkpoint                    ckpt-200.index                fine_tuned_40000/
ckpt-195.data-00000-of-00001  ckpt-201.data-00000-of-00001  fine_tuned_45000/
ckpt-195.index                ckpt-201.index                fine_tuned_50000/
ckpt-196.data-00000-of-00001  eval/                         fine_tuned_55000/
ckpt-196.index                fine_tuned_10000/             fine_tuned_65000/
ckpt-197.data-00000-of-00001  fine_tuned_12500/             fine_tuned_70000/
ckpt-197.index                fine_tuned_15000/             fine_tuned_7500/
ckpt-198.data-00000-of-00001  fine_tuned_17500/             fine_tuned_75000/
ckpt-198.index                fine_tuned_20000/             fine_tuned_80000/
ckpt-199.data-00000-of-00001  fine_tuned_25000/             fine_tuned_85000/
ckpt-199.index                fine_tuned_30000/             fine_tuned_90000/
ckpt-200.data-00000-of-00001  fine_tuned_35000/             train/


In [28]:
#run conversion script
import re
import numpy as np

output_directory = model_dir+'fine_tuned_100000'

#place the model weights you would like to export here
last_model_path = model_dir
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/content/drive/MyDrive/COLAB\ PROJECTS/furniture-detection/effdet-d0/
I0206 08:26:22.602737 139787000223616 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet EfficientNet backbone version: efficientnet-b0
I0206 08:26:22.602929 139787000223616 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num filters: 64
I0206 08:26:22.603002 139787000223616 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num iterations: 3
I0206 08:26:22.611336 139787000223616 efficientnet_model.py:147] round_filter input=32 output=32
I0206 08:26:22.639410 139787000223616 efficientnet_model.py:147] round_filter input=32 output=32
I0206 08:26:22.639535 139787000223616 efficientnet_model.py:147] round_filter input=16 output=16
I0206 08:26:22.699453 139787000223616 efficientnet_model.py:147] round_filter input=16 output=16
I0206 08:26:22.699582 139787000223616 efficientnet_model.py:147] round_filter input=24 output=24
I0206 08:26:22.852805 139787000223616 efficientnet_m

In [29]:
%ls {output_directory+"/saved_model"}

assets/  saved_model.pb  variables/


# Inference Model

In [30]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "/content/drive/MyDrive/COLAB PROJECTS/furniture-detection/effdet-d0/fine_tuned_100000/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

Done! Took 24.21548342704773 seconds


In [31]:
category_index = label_map_util.create_category_index_from_labelmap("/content/dataset/label_map.pbtxt",
                                                                    use_display_name=True)

In [36]:
import os
import pathlib
import tensorflow as tf
import warnings

# url = "https://images.ctfassets.net/fvljq824t995/217VLYSTHoNrbOYj8aLor9/63f77d5a8d0cb4e63be6113432f443eb/Industrial_living_room_furniture.jpg"
# url = "https://b2cfurniture.com.au/pub/media/catalog/product/cache/3fb871f48f7af5e44260f2d9fd3932a9/c/r/cruz-5-pce-home-furniture-package-timber-living-room-furniture-set.jpg"
# url = "https://www.comfortfurniture.com.sg/blog/wp-content/uploads/2019/08/featured-choices-of-living-room-furniture-showroom-comfort-furniture.jpg"
url = "https://www.ifamy.com/wp-content/uploads/2019/08/13%E5%89%AF%E6%9C%AC.jpg"

image_path = tf.keras.utils.get_file(fname="test2.jpg",
                                    origin=url,
                                    untar=False)
image_path = pathlib.Path(image_path)

180224/177428 [==============================] - 1s 4us/step


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.35,
        agnostic_mode=False)

plt.figure(figsize=(28, 28))
plt.imshow(image_np_with_detections)
print('Done')
plt.show()